In [ ]:
! pip install -U sentence-transformers

In [ ]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer,util

In [ ]:
class Answerer:
  def __init__(self, model):
    self.model = model
    self.corpus = []
    self.sentence_embeddings = []
    self.clusters = []
  
  def update(self, corpus):
    self.corpus = corpus
    self.sentence_embeddings = self.model.encode(corpus)
  
  def cluster(self, min_community_size=1, threshold=0.45):
    clusters = util.community_detection(self.sentence_embeddings, min_community_size=min_community_size, threshold=threshold)
    for i, cluster in enumerate(clusters):
        print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
        for sentence_id in cluster[0:3]:
            print("\t", self.corpus[sentence_id])
        print("\t", "...")
        for sentence_id in cluster[-3:]:
            print("\t", self.corpus[sentence_id])
    
    self.clusters = clusters

  def find_query_cluster(self, query_sentence, k=3):
    query_embedding = self.model.encode(query_sentence)

    # KNN
    similarity = util.cos_sim(query_embedding, self.sentence_embeddings)
    topk_idx = np.argsort(similarity)[:, -k:][0]
    cluster_count = np.zeros(len(self.clusters))
    for idx in topk_idx:
      cluster_idx = find_cluster(idx, self.clusters)
      if cluster_idx != -1:
        cluster_count[cluster_idx] += 1
      
    return cluster_count.argmax()
    
    



In [ ]:
def find_cluster(idx, clusters):
  for i, cluster in enumerate(clusters):
    if idx in cluster:
      return i
  return -1

In [ ]:
corpus = ["Any Questions?",
          "질문있나요?",
          "질문 있는 사람?",
          "물어볼 거 있는 사람?",
          "Something to ask?",
          "Are you there, Wonsuhk?",
          "Wonsuhk! Answer!"]

query_sentence = "질문?"

model = SentenceTransformer('all-MiniLM-L6-v2')
answerer = Answerer(model)
answerer.update(corpus)
answerer.cluster()
answerer.find_query_cluster(query_sentence)